# 何故regexで国名抽出するときにNERで前処理する必要があるか？

Date: 2023/12/06

In [6]:
import spacy
import re

nlp = spacy.load('ja_core_news_sm')

In [2]:
TEXT1 = '明日は本社で日本語を話せるドイツ人に合う'
TEXT2 = '明日は本社でドイツの海外現地法人から来た出張者と日本の話をする'
pattern = r'日本|ドイツ'

In [3]:
doc = nlp(TEXT1)
for ent in doc.ents:
    print(ent.text, ent.label_)
    match = re.match(pattern, ent.text)
    if match:
        print(match)

明日 PERSON
日本語 LANGUAGE
<re.Match object; span=(0, 2), match='日本'>
ドイツ人 NORP
<re.Match object; span=(0, 3), match='ドイツ'>


In [4]:
doc = nlp(TEXT2)
for ent in doc.ents:
    print(ent.text, ent.label_)
    match = re.match(pattern, ent.text)
    if match:
        print(match)

明日 PERSON
ドイツ GPE
<re.Match object; span=(0, 3), match='ドイツ'>
日本 GPE
<re.Match object; span=(0, 2), match='日本'>


In [5]:
for doc in nlp.pipe([TEXT1, TEXT2]):
    print(f'---{doc.text}')
    for ent in doc.ents:
        print(ent.text, ent.label_)
        if ent.label_ in ('GPE', 'ORG', 'LOC'):
            match = re.match(pattern, ent.text)
            if match:
                print(match, ent.label_)

---明日は本社で日本語を話せるドイツ人に合う
明日 PERSON
日本語 LANGUAGE
ドイツ人 NORP
---明日は本社でドイツの海外現地法人から来た出張者と日本の話をする
明日 PERSON
ドイツ GPE
<re.Match object; span=(0, 3), match='ドイツ'> GPE
日本 GPE
<re.Match object; span=(0, 2), match='日本'> GPE
